In [ ]:
pip install tensorflow

  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
Note: you may need to restart the kernel to use updated packages

In [2]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

from PIL import Image


In [ ]:
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1,
                          alpha=1)
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

In [ ]:
model_architecture = 'vgg16'
model = model_picker(model_architecture)

58889256/58889256 [==============================] - 8s 0us/step


In [ ]:

def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [ ]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [ ]:
root_dir = r'C:\Users\Admin\Downloads\101_ObjectCategories'
filenames = sorted(get_file_list(root_dir))

In [ ]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict_generator(generator, num_epochs)
end_time = time.time()

Found 8677 images belonging to 101 classes.


C:\Users\Admin\AppData\Local\Temp\ipykernel_13088\1637432355.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  feature_list = model.predict_generator(generator, num_epochs)


In [ ]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 512)
Time taken in sec =  954.0315155982971


In [ ]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size = 64,
                                                    shuffle = True,
                                                    seed = 42,
                                                    class_mode= 'categorical')

Found 8677 images belonging to 101 classes.


In [ ]:
def model_maker():
  base_model = VGG16(include_top = False,
                        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
  for layer in base_model.layers[:]:
    layer.trainable = False
  input = Input(shape = (IMG_WIDTH, IMG_HEIGHT, 3))
  custom_model = base_model(input)
  custom_model = GlobalAveragePooling2D()(custom_model)
  custom_model = Dense(64, activation='relu')(custom_model)
  custom_model = Dropout(0.5)(custom_model)
  predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
  return Model(inputs=input, outputs=predictions)

In [ ]:
model_finetuned = model_maker()
model_finetuned.compile(loss = 'categorical_crossentropy',
                        optimizer=tensorflow.keras.optimizers.Adam(0.001),
                        metrics=['acc'])
model_finetuned.fit(
    train_generator,
    steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs = 10)

Epoch 1/10
136/136 [==============================] - 946s 7s/step - loss: 4.1168 - acc: 0.2217
Epoch 2/10
136/136 [==============================] - 990s 7s/step - loss: 2.8775 - acc: 0.3689
Epoch 3/10
136/136 [==============================] - 1001s 7s/step - loss: 2.3475 - acc: 0.4586
Epoch 4/10
136/136 [==============================] - 941s 7s/step - loss: 1.9506 - acc: 0.5307
Epoch 5/10
136/136 [==============================] - 972s 7s/step - loss: 1.7030 - acc: 0.5670
Epoch 6/10
136/136 [==============================] - 1059s 8s/step - loss: 1.5316 - acc: 0.5984
Epoch 7/10
136/136 [==============================] - 943s 7s/step - loss: 1.3793 - acc: 0.6302
Epoch 8/10
136/136 [==============================] - 961s 7s/step - loss: 1.2623 - acc: 0.6596
Epoch 9/10
136/136 [==============================] - 973s 7s/step - loss: 1.2041 - acc: 0.6679
Epoch 10/10
136/136 [==============================] - 974s 7s/step - loss: 1.1230 - acc: 0.6900


In [ ]:
model_finetuned.save(r'model-vgg16.h5')

In [ ]:
model_architecture = 'resnet'
model = model_picker(model_architecture)

In [ ]:

def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [ ]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict_generator(generator, num_epochs)
end_time = time.time()

In [ ]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

In [ ]:
def model_maker():
  base_model = ResNet50(include_top = False,
                        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
  for layer in base_model.layers[:]:
    layer.trainable = False
  input = Input(shape = (IMG_WIDTH, IMG_HEIGHT, 3))
  custom_model = base_model(input)
  custom_model = GlobalAveragePooling2D()(custom_model)
  custom_model = Dense(64, activation='relu')(custom_model)
  custom_model = Dropout(0.5)(custom_model)
  predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
  return Model(inputs=input, outputs=predictions)

In [ ]:
model_finetuned = model_maker()
model_finetuned.compile(loss = 'categorical_crossentropy',
                        optimizer=tensorflow.keras.optimizers.Adam(0.001),
                        metrics=['acc'])
model_finetuned.fit(
    train_generator,
    steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs = 10)

Epoch 1/10
136/136 [==============================] - 426s 3s/step - loss: 2.8964 - acc: 0.3710
Epoch 2/10
136/136 [==============================] - 435s 3s/step - loss: 1.6128 - acc: 0.6031
Epoch 3/10
136/136 [==============================] - 425s 3s/step - loss: 1.1905 - acc: 0.6864
Epoch 4/10
136/136 [==============================] - 423s 3s/step - loss: 1.0151 - acc: 0.7235
Epoch 5/10
136/136 [==============================] - 439s 3s/step - loss: 0.9006 - acc: 0.7548
Epoch 6/10
136/136 [==============================] - 438s 3s/step - loss: 0.8034 - acc: 0.7707
Epoch 7/10
136/136 [==============================] - 435s 3s/step - loss: 0.7506 - acc: 0.7915
Epoch 8/10
136/136 [==============================] - 458s 3s/step - loss: 0.6916 - acc: 0.8018
Epoch 9/10
136/136 [==============================] - 440s 3s/step - loss: 0.6594 - acc: 0.8119
Epoch 10/10
136/136 [==============================] - 447s 3s/step - loss: 0.6185 - acc: 0.8165


In [ ]:
model_finetuned.save(r'model-resnet.h5')

In [ ]:
def get_extract_model():
    resnet_model = load_model(r'model-vgg16.h5')
    extract_model = Model(inputs=resnet_model.inputs, outputs = resnet_model.output)
    return extract_model

def image_preprocess(img):
    img = img.resize((224,224))
    img = img.convert("RGB")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def extract_vector(model, image_path):
    print("Xu ly : ", image_path)
    img = Image.open(image_path)
    img_tensor = image_preprocess(img)

    vector = model.predict(img_tensor)[0]

    vector = vector / np.linalg.norm(vector)
    return vector





data_folder = r'C:\Users\Admin\Downloads\101_ObjectCategories'
list_data = [f for f in os.listdir(data_folder) if os.listdir(data_folder)]
model = get_extract_model()

vectors = []
paths = []

for image_path in os.listdir(data_folder):
    onlyfiles = [f for f in os.listdir(os.path.join(data_folder,image_path)) if os.path.join(data_folder, image_path, f)]
    for file in onlyfiles:
        image_path_full = os.path.join(data_folder, image_path, file)

        image_vector = extract_vector(model, image_path_full)
        print(image_path)
        vectors.append(image_vector)
        paths.append(image_path_full)


vector_file = "vectors_resnet.pkl"
path_file = "paths_resnet.pkl"

pickle.dump(vectors, open(vector_file, "wb"))
pickle.dump(paths, open(path_file, "wb"))

Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0001.jpg
1/1 [==============================] - 1s 709ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0002.jpg
1/1 [==============================] - 0s 326ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0003.jpg
1/1 [==============================] - 0s 326ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0004.jpg
1/1 [==============================] - 1s 569ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0005.jpg
1/1 [==============================] - 0s 327ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0006.jpg
1/1 [==============================] - 0s 304ms/step
accordion
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\accordion\image_0007.jpg
1/1 [==============================] - 0s 243ms/step
accordion

1/1 [==============================] - 0s 311ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0004.jpg
1/1 [==============================] - 0s 299ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0005.jpg
1/1 [==============================] - 0s 306ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0006.jpg
1/1 [==============================] - 0s 303ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0007.jpg
1/1 [==============================] - 0s 333ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0008.jpg
1/1 [==============================] - 0s 306ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0009.jpg
1/1 [==============================] - 0s 330ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0010.jpg

1/1 [==============================] - 0s 334ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0061.jpg
1/1 [==============================] - 0s 262ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0062.jpg
1/1 [==============================] - 0s 322ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0063.jpg
1/1 [==============================] - 0s 288ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0064.jpg
1/1 [==============================] - 0s 293ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0065.jpg
1/1 [==============================] - 0s 268ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0066.jpg
1/1 [==============================] - 0s 323ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0067.jpg

1/1 [==============================] - 0s 246ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0118.jpg
1/1 [==============================] - 0s 299ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0119.jpg
1/1 [==============================] - 0s 282ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0120.jpg
1/1 [==============================] - 0s 320ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0121.jpg
1/1 [==============================] - 0s 280ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0122.jpg
1/1 [==============================] - 0s 345ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0123.jpg
1/1 [==============================] - 0s 281ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0124.jpg

1/1 [==============================] - 0s 273ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0175.jpg
1/1 [==============================] - 0s 314ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0176.jpg
1/1 [==============================] - 0s 377ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0177.jpg
1/1 [==============================] - 0s 340ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0178.jpg
1/1 [==============================] - 0s 311ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0179.jpg
1/1 [==============================] - 0s 314ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0180.jpg
1/1 [==============================] - 0s 322ms/step
airplanes
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\airplanes\image_0181.jpg

...

1/1 [==============================] - 1s 546ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0005.jpg
1/1 [==============================] - 1s 550ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0006.jpg
1/1 [==============================] - 1s 568ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0007.jpg
1/1 [==============================] - 1s 643ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0008.jpg
1/1 [==============================] - 1s 560ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0009.jpg
1/1 [==============================] - 1s 697ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\water_lilly\image_0010.jpg
1/1 [==============================] - 1s 614ms/step
water_lilly
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategorie

1/1 [==============================] - 1s 547ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0024.jpg
1/1 [==============================] - 1s 630ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0025.jpg
1/1 [==============================] - 1s 534ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0026.jpg
1/1 [==============================] - 1s 539ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0027.jpg
1/1 [==============================] - 1s 640ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0028.jpg
1/1 [==============================] - 1s 573ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\image_0029.jpg
1/1 [==============================] - 1s 508ms/step
wheelchair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wheelchair\

1/1 [==============================] - 1s 597ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0022.jpg
1/1 [==============================] - 1s 599ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0023.jpg
1/1 [==============================] - 1s 611ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0024.jpg
1/1 [==============================] - 1s 598ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0025.jpg
1/1 [==============================] - 1s 565ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0026.jpg
1/1 [==============================] - 1s 583ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0027.jpg
1/1 [==============================] - 1s 559ms/step
wild_cat
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\wild_cat\image_0028.jpg
1/1 [========

1/1 [==============================] - 1s 645ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0043.jpg
1/1 [==============================] - 1s 640ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0044.jpg
1/1 [==============================] - 1s 776ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0045.jpg
1/1 [==============================] - 1s 678ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0046.jpg
1/1 [==============================] - 1s 652ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0047.jpg
1/1 [==============================] - 1s 677ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\windsor_chair\image_0048.jpg
1/1 [==============================] - 1s 668ms/step
windsor_chair
Xu ly :  C:\Users\Admin\Dow

1/1 [==============================] - 1s 662ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0006.jpg
1/1 [==============================] - 1s 699ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0007.jpg
1/1 [==============================] - 1s 673ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0008.jpg
1/1 [==============================] - 1s 656ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0009.jpg
1/1 [==============================] - 1s 670ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0010.jpg
1/1 [==============================] - 1s 681ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0011.jpg
1/1 [==============================] - 1s 691ms/step
yin_yang
Xu ly :  C:\Users\Admin\Downloads\101_ObjectCategories\yin_yang\image_0012.jpg
1/1 [========

In [3]:
def get_extract_model():
    vgg16_model = load_model(r'model-vgg16.h5')
    extract_model = Model(inputs=vgg16_model.inputs, outputs = vgg16_model.output)
    return extract_model


def image_preprocess(img):
    img = img.resize((224,224))
    img = img.convert("RGB")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def extract_vector(model, image_path):
    print("Xu ly : ", image_path)
    img = Image.open(image_path)
    img_tensor = image_preprocess(img)

    vector = model.predict(img_tensor)[0]
    
    vector = vector / np.linalg.norm(vector)
    return vector


search_image = r"F:\BAI TAP\New folder\MiAI_Image_Search\testimage\images (1).jpg"


model = get_extract_model()


search_vector = extract_vector(model, search_image)


vectors = pickle.load(open(r"C:\Users\Admin\vectors_resnet.pkl","rb"))
paths = pickle.load(open(r"C:\Users\Admin\paths_resnet.pkl","rb"))


distance = np.linalg.norm(vectors - search_vector, axis=1)

K = 4
ids = np.argsort(distance)[:K]


nearest_image = [(paths[id], distance[id]) for id in ids]


import matplotlib.pyplot as plt

axes = []
grid_size = int(math.sqrt(K))
fig = plt.figure(figsize=(10,5))


for id in range(K):
    draw_image = nearest_image[id]
    axes.append(fig.add_subplot(grid_size, grid_size, id+1))

    axes[-1].set_title(draw_image[1])
    plt.imshow(Image.open(draw_image[0]))

fig.tight_layout(pad=0.5)
plt.show()

OSError: ignored